<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 13. Interfaces, Protocols, and ABCs

Program to an interface, not an implementation

Since python 3.8, we have four ways to define an interfaces:
 - Duck typing: Python's default approach
 - Goose typing: supported by abstract base classes (ABCs) which rely on runtime checks of objects against ABCs.
 - Static typing: traditional approach of statically-typed langauges like C and Java
 - Static duck typing: Go langauge approach; supported by subclasses of `typing.Protocol`

## Two Kinds of Protocols

An object protocol specifies methods which an object must provide to fulfill a role.

In [ ]:
class Vowels:
  def __getitem__(self, i):
    return 'AEIOU'[i]

In [ ]:
v = Vowels()
v[0]

'A'

In [ ]:
v[-1]

'U'

In [ ]:
for c in v:
  print(c)

A
E
I
O
U


In [ ]:
'E' in v

True

In [ ]:
'Z' in v

False

Implementing `__getitem__` is enough to allow retrieving items by index, and also to support iteration and the `in` operator.

## Programming Ducks

Sequence and iterable protocols

In summary, given the importance of sequence-like data structures, Python manages to make iteration and the `in` operator work by invoking `__getitem__` when `__iter__` and `__contains` are unavailable

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

The original `FrenchDeck` does not subclass `abc.Sequence` either, but it does implement both methods of the sequence protocol: `__getitem__` and `__len__`

### Why static type checkers have no chance of dealing with them?

## Monkey Patching: Implementing a Protocol at Runtime

In [ ]:
from random import shuffle

l = list(range(10))
shuffle(l)

In [ ]:
l

[7, 5, 9, 0, 1, 4, 2, 8, 3, 6]

In [ ]:
deck = FrenchDeck()

In [ ]:
shuffle(deck)

In [ ]:
def set_card(deck, position, card):
  deck._cards[position] = card

In [ ]:
FrenchDeck.__setitem__ = set_card

In [ ]:
shuffle(deck)

In [ ]:
deck[:5]

[Card(rank='2', suit='hearts'),
 Card(rank='6', suit='hearts'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='A', suit='hearts')]

## Defensive Programming and "Fail Fast"

In [ ]:
def __init__(self, iterable):
  # don't type check iterable is iterable
  # just use list to see whether iterable is iterable
  self._balls = list(iterable)


## Goose Typing

`isinstance(obj, cls)` is now just fine as long as `cls` is an abstract base class--in other words, `cls`'s metaclass is `abc.ABCMeta`

In [ ]:
class Struggle:
  def __len__(self): return 23

In [ ]:
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [ ]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

# not check for the implementation of the abstract methods at import time
# but only at runtime when we actually try to instantiate FrenchDeck2
class FrenchDeck2(abc.MutableSequence):
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

  def __setitem__(self, position, value): # all we need for enabling shuffling
    self._cards[position] = value

  def __delitem__(self, position): # but subclassing MutableSequence forces us to implment __delitem__
    del self._cards[position]

  def insert(self, position, value):
    self._cards.insert(position, value)

#

The use of `isinstance` and `issubclass` becomes more acceptable if you are checking agains ABCs instead of concrete classes. If used with concrete classes, type checks limit polymorphism.

In [ ]:
from collections.abc import Sequence
issubclass(FrenchDeck2, Sequence)

True

In [ ]:
deck = FrenchDeck2()

In [ ]:
for i in reversed(deck):
  print(i)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

To use ABCs well, you need to know what's available. We'll review the `collections` ABCs next.

## ABCs in the Standard Library

The only reliable way to determine whether an object is iterable is to call `iter(obj)`. Same as `hash(obj)`.

Defining and Using an ABC

In [1]:
# tombola.py

import abc

class Tombola(abc.ABC):

  @abc.abstractmethod
  def load(self, iterable):
    """Add items from an iterable"""

  @abc.abstractmethod
  def pick(self):
    """ Remove item at random, returning it

    This method should raise `LookupError` when the instance is empty
    """

  def loaded(self):
    return bool(self.inspect())

  def inspect(self):
    items = []
    while True:
      try:
        items.append(self.pick())
      except LookupError:
        break
    self.load(items)
    return tuple(items)



In [ ]:
class Fake(Tombola):
  def pick(self):
    return 13

In [ ]:
Fake

__main__.Fake

In [ ]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

# Subclassing an ABC

In [2]:
import random

class BingoCage(Tombola):
  def __init__(self, items):
    self._randomizer = random.SystemRandom()
    self._items = []
    self.load(items)

  def load(self, items):
    self._items.extend(items)
    self._randomizer.shuffle(self._items)

  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty BingoCage')

  def __call__(self):
    self.pick()

In [3]:
import random

class LottoBlower(Tombola):
  def __init__(self, iterable):
    self._balls = list(iterable)

  def load(self, iterable):
    self._balls.extend(iterable)

  def pick(self):
    try:
      position = random.randrange(len(self._balls))
    except ValueError:
      raise LookupError('pick from empty LottoBlower')

    return self._balls.pop(position)

  def loaded(self):
    return bool(self._balls)

  def inspect(self):
    return tuple(self._balls)

We now come to the crucial dynamic feature of goose typing: declaring virtual subclasses with the `register` method.

## A virtual subclass of an ABC

The registered class becomes a virtual subclass of ABC, and will be recognized as such by `issubclass` but it does not inherit any methods or attributes from the ABC.

In [4]:
from random import randrange

@Tombola.register # Tombolist is registered as a virtual subclass of Tombola
class TomboList(list):
  def pick(self):
    if self:
      position = randrange(len(self))
      return self.pop(position)
    else:
      raise LookupError('pop from empty TomboList')

  load = list.extend

  def loaded(self):
    return bool(self)

  def inspect(self):
    return tuple(self)


In [ ]:
issubclass(TomboList, Tombola)

True

In [ ]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [ ]:
TomboList.__mro__

(__main__.TomboList, list, object)

# Structural Typings with ABCs

Structural typing is about looking at the structure of an object's public interface to determine its  type: an object is consistent-with a type if it implements the methods defined in the type.

In [5]:
class Struggle:
  def __len__(self): return 23

In [6]:
from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [7]:
issubclass(Struggle, abc.Sized)

True

## Static Protocols

- illustrate static protocols with two simple examples
- discuss numeric ABCs and protocols


### The Typed double function
 example of showing how a static protocol makes it possible to annotate and type check the `double()` function


In [8]:
def double(x):
  return x * 2

In [9]:
double(1.5)

3.0

In [10]:
double('A')

'AA'

In [11]:
double([10, 20, 30])

[10, 20, 30, 10, 20, 30]

In [12]:
from fractions import Fraction

In [13]:
double(Fraction(2, 5))

Fraction(4, 5)

Before static protocols were introduced, there was no practical way to add type hints to `double` without limiting its possible uses.

In [14]:
# double_protocol.py

from typing import TypeVar, Protocol

T = TypeVar('T')

class Repeatable(Protocol):
  # self parameter is usually not annotaed
  # but we use T to make sure that the result type
  # is the same as the type of self
  # repeat_count: limited to int
  def __mul__(self: T, repeat_count: int) -> T: ...

# RT type variable is bounded by the Repeatable protocol
# type checker will require that the actual type implements
# Repeatable
RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
  return x * 2

In [ ]:
# typing.SupportsComplex protocol src code
@runtime_checkable
class SupportsComplex(Protocol):
  __slots___ = ()

  @abstractmethod
  def __complex__(self) -> complex:
    pass

In [15]:
from typing import SupportsComplex
import numpy as np

In [16]:
c64 = np.complex64(3+4j)

In [17]:
isinstance(c64, complex)

False

In [18]:
isinstance(c64, SupportsComplex)

True

In [19]:
c = complex(c64)
c

(3+4j)

In [20]:
isinstance(c, SupportsComplex)

False

In [23]:
import numbers

In [24]:
isinstance(c, numbers.Complex)

True

In [25]:
isinstance(c64, numbers.Complex)

True

## Limitations of Runtime Protocol Checks

In [26]:
import sys
sys.version

'3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'

In [37]:
# vector2d_v4.py
from array import array
import math

class Vector2d:
  __match_args__ = ('x', 'y') # class attribute listing the instance attributes in the order
                              # they will be used for positional pattern matching
  __slots__ = ('__x', '__y')

  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes


  def __init__(self, x, y):
    self.__x = float(x) # converting x to float catches errors early
    self.__y = float(y)

  @property
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  # classmethod decorator modifies a method
  # so it can be called directly on a class
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self) -> float:
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

  def __hash__(self):
    return hash((self.x, self.y))

  def __complex__(self) -> complex:
    return complex(self.x, self.y)

  @classmethod
  def fromcomplex(cls, datum: SupportsComplex):
    c = complex(datum)
    return cls(c.real, c.imag)

In [38]:
from typing import SupportsComplex, SupportsAbs

In [39]:
v = Vector2d(3, 4)
isinstance(v, SupportsComplex)

True

In [33]:
isinstance(v, SupportsAbs)

True

In [34]:
complex(v)

(3+4j)

In [35]:
abs(v)

5.0

In [36]:
Vector2d.fromcomplex(3+4j)

Vector2d(3.0, 4.0)

For better static coverage and error reporting, `__abs__`, `__complex__`, and `fromcomplex` methods should get type hints

# Designing a Static Protocol

Single-method protocols make static duck typing more useful and flexible.

In [40]:
# randompick.py

from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
  def pick(self) -> Any: ...

In [41]:
# randompick_test.py

import random
from typing import Any, Iterable, TYPE_CHECKING

In [42]:
class SimplePicker:
  def __init__(self, items: Iterable) -> None:
    self._items = list(items)
    random.shuffle(self._items)

  def pick(self) -> Any:
    return self._items.pop()

In [43]:
def test_isinstance() -> None:
  popper: RandomPicker = SimplePicker([1])
  assert isinstance(popper, RandomPicker)

def test_item_type() -> None:
  items = [1, 2]
  popper = SimplePicker(items)
  item = popper.pick()
  assert item in items
  if TYPE_CHECKING:
    reveal_type(item) # this line generates a note in the mypy note
  assert isinstance(item, int)

In [44]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 45.3 MB/s eta 0:00:00


In [45]:
!mypy randompick_test.py

randompick_test.py:26: note: Revealed type is "Any"
Success: no issues found in 1 source file


## Extending a Protocol

In [ ]:
from typing import Protocol, runtime_checkable
from randompick import RandomPicker

@runtime_checkable
class LoadableRandomPicker(RandomPicker, Protocol):
  def load(self, Iterable) -> None: ...